In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2" #näkyvät gput
#kaikki GPU:iden virrankulutus rajoitettu 215 W.
#GPU 0: NVIDIA GeForce RTX 3090 (UUID: GPU-6301e069-f0d5-8843-e69c-5433fe790fb2)
#GPU 1: NVIDIA GeForce RTX 3090 (UUID: GPU-ac6d41e1-946c-9d43-8532-e04c96f85d6c)
#GPU 2: NVIDIA GeForce RTX 3090 Ti (UUID: GPU-1fdc37ad-7cdd-78b2-7e78-450b86be0512)
#   index  name                 id
#0, NVIDIA GeForce RTX 3090,    00000000:04:00.0
#1, NVIDIA GeForce RTX 3090,    00000000:0B:00.0
#2, NVIDIA GeForce RTX 3090 Ti, 00000000:0C:00.0

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
)

from peft import LoraConfig, get_peft_model
from trl import DPOTrainer
import torch

#debuggausta varten, kaikki gput eivät aina näkyneet syistä X
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"number of gpus: {num_gpus}")

    for i in range(num_gpus):
        print(f"gpu {i}: {torch.cuda.get_device_name(i)}")
        print(f"  vram: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.2f} GB")
        print(f"  cuda: {torch.cuda.get_device_properties(i).major}.{torch.cuda.get_device_properties(i).minor}")
else:
    print("no gpus")

In [ ]:
base_model = "mistralai/Mistral-7B-v0.1" #valmiiksi koulutettu mistral malli huggingfacesta
new_model = "MistralDraft"#uusi malli
dataset = load_dataset("unalignment/toxic-dpo-v0.2")['train']#datasetti jota käytetään koulutukseen.    
print(dataset)

In [ ]:
def format_prompt(example): #mistral malli vaatii oman muotoilun.
    return {
        "prompt": f"[INST] {example['prompt']} [/INST]",
        "chosen": example['chosen'], #hyväksytty vastaus,
        "rejected": example['rejected'] #hylätty vastaus.
    }

In [ ]:
formatted_dataset = dataset.map(format_prompt) #muotoillaan datasetti   

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model) #ladataan tokenizer.
tokenizer.pad_token = tokenizer.eos_token #EOS - end of sentence
tokenizer.padding_side = "right" #DPO koulutusta varten. Malli lukee tekstin vasemmalta oikealle, joten EOS tulee outputin loppuun


In [ ]:
peft_config = LoraConfig( #lora config.
    r=16, #rank
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"] #kohde moduulit
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map = "auto", #ladataan malli automaattisesti kaikille gpuille
    max_memory={0: "23GiB", 1: "23GiB", 2: "23GiB"},#vram rajoitetaan 23/24gb jotta muisti ei lopu kesken.
    trust_remote_code=True,
    torch_dtype=torch.bfloat16 #käytetään bf16 tyyppiä, fp16:n sijaan. (rtx 3090)
)
#model = prepare_model_for_kbit_training(model)  #4bit model, ei tarvita koska muistia riittää.
model = get_peft_model(model, peft_config) #käytetään LoRAA malliin.
model.config.use_cache

In [ ]:
ref_model = AutoModelForCausalLM.from_pretrained( #reference mallia tarvitaan DPO koulutukseen. mallina toimii sama base_model (mistral)
    base_model,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    max_memory={0: "23GiB", 1: "23GiB", 2: "23GiB"},
)

In [ ]:
import wandb
wandb.login() #koulutuksen metriikka kirjataan wandb:hen. https://wandb.ai/home

In [ ]:
#koulutusparametrit
training_args = TrainingArguments(
    per_device_train_batch_size=2, #batch
    gradient_accumulation_steps=1, #simuloi suurempaa batch kokoa, jätän 1 koska muistia riittää.
    gradient_checkpointing=True, #säästää vramia paljon.
    learning_rate=3e-5,
    lr_scheduler_type="cosine",
    max_steps=80, #koulutuksen askelmäärä.
    logging_steps=10, #lokia kerätään joka 10 askeleen välein.
    output_dir="./results", #uuden mallin sijainti
    optim="adamw_torch", #optimizer:AdamW
    warmup_steps=20,
    bf16=True,
    fp16=False, #fp16 ei käytetä, bf16 "stabiilimpi".
    remove_unused_columns=False,
    num_train_epochs=1, #ei tarvitse määrittää erikseen, max_steps määrittää tämän DPO koulutuksessa.
    save_strategy="no",
    report_to="wandb",
)

In [ ]:
dpo_trainer = DPOTrainer( #DPO trainer (direct preference optimization)
    model=model,
    ref_model=ref_model,
    args=training_args,
    beta=0.1, #tämä arvo säätää koulutuksen "voimakkuutta" mallille.
    train_dataset=formatted_dataset,
    tokenizer=tokenizer,
    max_prompt_length=1024, #max input pituus
    max_length=2048, #max output pituus
    generate_during_eval=False,
)

In [ ]:
dpo_trainer.train() # koulutus lopetettu 70. stepin kohdalla.

In [ ]:
model.save_pretrained("./mistral-dpo-lora-adapter")
tokenizer.save_pretrained("./mistral-dpo-lora-adapter") #tallennetaan koulutettu malli (ja sen tokenizer.)